In [1]:
import numpy as np

from human_terms_network import Human_Terms_Network
from dataset_load import *
from lr_baseline import LRBaseline

Using TensorFlow backend.


In [2]:
path = r"..\..\data\reviews_Amazon_Instant_Video_5.json.gz"

X, y = extract_review_amazon(path, 'reviewText')

y_label = y
# discard the review with 3. [1,2] = 0, [4,5] = 0
neutral_indices = np.where(y_label == 3)[0]
y_label[y_label<3] = 0
y_label[y_label>3] = 1

X_discarded = np.delete(X,neutral_indices)
y_discarded = np.delete(y_label, neutral_indices)

from sklearn.model_selection import train_test_split

X_train_split, X_test_split, y_train_amazon, y_test_amazon = train_test_split(
    X_discarded, y_discarded)

X_train_corpus_update = update_corpus_contraction(X_train_split)
X_test_corpus_update = update_corpus_contraction(X_test_split)



(75, 2)
corpus update start
corpus update end

(75, 2)
corpus update start
corpus update end



In [3]:
word_list, connotation = load_unigrams('./amazon-video-unigrams-more.txt', X_train_corpus_update, y_train_amazon)

In [4]:
y_train_agreement, y_test_agreement = generate_appearance(X_train_corpus_update, X_test_corpus_update, 
                                                          word_list, connotation)

In [5]:
av_baseline = LRBaseline(X_train_corpus_update, X_test_corpus_update, y_train_amazon, y_test_amazon, 
                           human_terms=word_list)

In [6]:
# (train, test)

print(av_baseline.baseline())
print(av_baseline.baseline(penalty='l1'))
# print(av_baseline.human_terms_baseline())
# print(av_baseline.human_terms_baseline(penalty='l1'))

(0.9523963730569949, 0.9289617486338798)
(0.9507367227979274, 0.928354584092289)


In [7]:
len(word_list)

78

In [8]:
from sklearn.linear_model import LogisticRegression

In [9]:
clf = LogisticRegression(penalty='l1')
X_train = av_baseline.baseline_cv.transform(X_train_corpus_update)
X_test = av_baseline.baseline_cv.transform(X_test_corpus_update)

clf.fit(X_train, y_train_amazon)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [10]:
weight = clf.coef_.flatten()

In [11]:
word = av_baseline.baseline_cv.get_feature_names()

In [12]:
indices = np.argsort(weight)[::-1]

for i in indices[200:300]:
    print(word[i])

beautiful
wow
husband
media
ratings
office
both
human
standard
twist
large
enjoyable
robert
hear
true
own
begin
hopefully
know
investigation
eventually
bosch
lawyer
rich
mysteries
wish
acted
keep
prime
tale
easily
features
thanks
huge
engaged
darker
dad
relationships
battle
faces
currently
gotten
quite
opens
flick
likes
check
action
crazy
top
showed
cancelled
words
surprises
catch
sister
explain
finding
fresh
bit
our
including
continue
seasons
forward
upon
air
each
present
pick
coming
dr
six
since
real
e
detail
planet
life
compared
guys
officer
considering
missed
away
paul
times
police
slightly
five
seeing
complex
effects
regular
though
grew
matter
bang
winner
happen


In [13]:
X_train = av_baseline.baseline_cv.transform(X_train_corpus_update)
X_test = av_baseline.baseline_cv.transform(X_test_corpus_update)
y_train = y_train_amazon
y_test = y_test_amazon

In [14]:
amazon_htm = Human_Terms_Network(input_shape=X_train.shape[1],
                               human_terms_shape=len(word_list),
                                domain='amazon', optimizer='adam')

In [15]:
amazon_htm.set_data(X_train, X_test, y_train_agreement, y_test_agreement, y_train, y_test)

In [16]:
amazon_htm.train(epochs=10)

AttributeError: 'Human_Terms_Network' object has no attribute 'y_train_original'

In [ ]:
amazon_htm.test(reject=True)

## has not been synchronized

In [ ]:
amazon_htm.test(reject=False)

In [ ]:
concat_after_relu = Model(inputs=combined_model.input,
                          outputs=combined_model.get_layer(layer_name).output)
concat_output = concat_after_relu.predict([X_test[idx], np.reshape(y_test_agreement[idx], (1,y_test_agreement.shape[1]))])

indices = np.where(np.sum(concat_all, axis=1)!=0)
indices = indices[0]
len(indices)

